In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg') 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
sns.set(style='whitegrid')
import joblib
RANDOM_STATE = 42

In [ ]:
df=pd.read_csv(r'/kaggle/input/weather-type-classification/weather_classification_data.csv')

In [ ]:
df

In [ ]:
df.head(20)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isnull()

In [ ]:
df.isnull().sum()

In [ ]:
sns.boxplot(data=df, x='Weather Type', y='Humidity')
plt.title('Humidity Distribution by Weather Type')
plt.show()

In [ ]:
sns.violinplot(data=df, x='Weather Type', y='Temperature')
plt.title('Temperature Distribution by Weather Type')
plt.show()

In [ ]:
g = sns.FacetGrid(df, col='Weather Type')
g.map_dataframe(sns.scatterplot, x='Temperature', y='Humidity')
g.set_titles(col_template="{col_name}")
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoders = {}
for col in df.select_dtypes(include=['object', 'bool']).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le  

In [ ]:
df

# Logistic Regrision

In [ ]:
X = df.drop('Weather Type', axis=1)  
y = df['Weather Type']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size  = 0.25, random_state =0 )

In [ ]:
from sklearn.linear_model import LogisticRegression
classifer = LogisticRegression()
classifer.fit(X_train, y_train)

In [ ]:
y_pred = classifer.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(5,5))
sns.heatmap(cm,annot=True)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
from sklearn.metrics import  accuracy_score
cm = accuracy_score(y_test, y_pred)
cm

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test, y_pred)
acc=accuracy_score(y_pred,y_test)
acc

# Random forest algorithm 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(5,5))
sns.heatmap(cm,annot=True)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')



# Naive bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred)
acc

label Encoding for categorical weather

In [ ]:
season_encoder = LabelEncoder().fit(['Summer', 'Winter', 'Spring', 'Fall'])
location_encoder = LabelEncoder().fit(['inland', 'mountain', 'coastal'])
weather_encoder = LabelEncoder().fit(['Sunny', 'Rainy', 'Cloudy', 'Snowy'])



Saving trained model and encoder

In [ ]:
joblib.dump(model, 'weather_model.pkl')
joblib.dump(season_encoder, 'season_encoder.pkl')
joblib.dump(location_encoder, 'location_encoder.pkl')
joblib.dump(weather_encoder, 'weather_encoder.pkl')

print("file is saved!")

Full Weather prediction and Agricultural Recommendation

In [ ]:
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder

def get_float_input(prompt):
    while True:
        try:
            return float(input(prompt))
        except ValueError:
            print("Error: Please enter a valid number.")

def generate_recommendation(prediction, crop_type, weather_data):
    temp = weather_data['Temperature']
    precipitation = weather_data['Precipitation']
    hum = weather_data['Humidity']
    
    recommendations = {
        'Sunny': {
            'wheat': f"Sunny ({temp}°C): Increase wheat irrigation by 20% in morning",
            'corn': "Sunny: Reduce corn watering",
            'default': "Sunny weather: Normal irrigation recommended"
        },
        'Rainy': {
            'heavy': f"Heavy rain ({precipitation}mm): Avoid planting today",
            'light': "Light rain: Proceed with caution",
            'default': "Rainy: Monitor field drainage"
        },
        'Cloudy': {
            'default': "Cloudy: Reduce watering by 10%"
        },
        'Snowy': {
            'default': "Snowy: Use greenhouses to protect crops"
        }
    }
    
    if prediction == 'Rainy':
        if precipitation > 10:
            return recommendations['Rainy'].get('heavy')
        else:
            return recommendations['Rainy'].get('light')
    
    return recommendations.get(prediction, {}).get(crop_type.lower(), 
           recommendations.get(prediction, {}).get('default', "No specific recommendation"))

def main():
    try:
        model = joblib.load('weather_model.pkl')
        season_enc = joblib.load('season_encoder.pkl')
        loc_enc = joblib.load('location_encoder.pkl')
        weather_enc = joblib.load('weather_encoder.pkl')
    except FileNotFoundError:
        print("Model files not found! Please run training script first.")
        return

    print("\n=== Agricultural Advisory System ===")
    
    print("\nEnter weather data:")
    temp = get_float_input("Temperature (°C): ")
    wind = get_float_input("Wind speed (km/h): ")
    hum = get_float_input("Humidity (%): ")
    precipitation = get_float_input("Precipitation (mm): ")
    cloud = get_float_input("Cloud cover (%): ")
    pressure = get_float_input("Atmospheric pressure (hPa): ")
    uv = get_float_input("UV index: ")
    vis = get_float_input("Visibility (km): ")
    
    print(f"\nSelect season from: {list(season_enc.classes_)}")
    season = input("Season: ").strip()
    
    print(f"\nSelect location from: {list(loc_enc.classes_)}")
    location = input("Location: ").strip()
    
    crop = input("\nCrop type (e.g. wheat, corn): ").strip().lower()
    
    season_encoded = season_enc.transform([season])[0]
    loc_encoded = loc_enc.transform([location])[0]
    
    weather_data = {
        'Temperature': temp,
        'Precipitation': precipitation,
        'Humidity': hum
    }
    
    X_new = np.array([[
        temp, wind, hum, precipitation, cloud,
        pressure, uv, season_encoded, vis, loc_encoded
    ]])
    
    try:
        pred_num = model.predict(X_new)[0]
        weather = weather_enc.inverse_transform([pred_num])[0]
        
        print("\n=== Results ===")
        print(f"Predicted weather: {weather}")
        print("Recommendation:", generate_recommendation(weather, crop, weather_data))
    
    except Exception as e:
        print(f"Prediction error: {e}")

main()